## VMAF

In [ ]:
import subprocess
c_vmaf = f"ffmpeg -f rawvideo -r 50 -video_size 3840x2160 -i test_vids/downsampled/smithy_3840.yuv \
        -i test_vids/encoded/smithyAVC_3840_cf1.mp4 \
        -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                 [1:v]scale=3840x2160:flags=bicubic, setpts=PTS-STARTPTS[dist];\
                 [dist][ref]libvmaf=log_path=rd_results/vmaf_logs.json:log_fmt=json\" \
        -f null -"
subprocess.call(c_vmaf, shell=True)